In [15]:
from collections import defaultdict
from scipy.io import wavfile
from textgrid import TextGrid as TG
from textgrid import Interval
from tqdm import tqdm

import contextlib
import os
import wave

In [1]:
break # Old, simplified vesion. Do not use anymore.

root_folder = '../../allwavs/allvowl'

# Read all input files
all_files = os.listdir(root_folder)
all_lower_files = [file.lower() for file in all_files]
corpus = defaultdict(list)
final_corpus = {}

variant = True

# Keep the name of the speaker (without additional information) as the key of each file
for file in all_files:
    file_l = file.lower().replace('_1', '').replace('cut', '').replace('-corrected', '')
    if file_l.endswith('wav'):
        corpus[file_l[:-4]].append(file)
    if file_l.endswith('textgrid'):
        corpus[file_l[:-9]].append(file)

for k, v in corpus.items():
    # Only keep speakers for which we have both audio and transcriptions
    if not all(not x.lower().endswith('textgrid') for x in v) or all(not x.lower().endswith('wav') for x in v):
        wavs = [file for file in v if file.lower().endswith('wav')]
        
        # If multiple files are available, keep only the ones with 'cut' in the name
        if len(wavs) > 1:
            wavs = [x for x in wavs if 'cut' in x]
        wav = wavs[0]
        
        # If multiple transcription files, choose which one to keep
        textgrids = [file for file in v if file.lower().endswith('textgrid')]
        if len(textgrids) > 1:
            if variant:
                textgrids = [x for x in textgrids if '_1' in x]
            else:
                textgrids = [x for x in textgrids if 'mono.' in x]
            # Exception: One speaker has only one type of transcription
            if k == 'om1_mono':
                textgrids = ['om1_mono.TextGrid']
        textgrid = textgrids[0]
        final_corpus[k] = [wav, textgrid]
    else:
        print(k, v)

SyntaxError: 'break' outside loop (3336785297.py, line 1)

In [ ]:
root_folder = '../../allwavs/allvowl/corrected'

all_files = os.listdir(root_folder)
final_corpus = {}

variant = True

# Generate a dictionary of speaker - (audio file, transcription file) key-values
for file in all_files:
    if file.endswith('wav'):
        file_l = file.replace('cut', '')
        final_corpus[file_l.replace('.wav', '')] = (file, file.replace('wav', 'textgrid'))

In [22]:
marks = set()

# Set of all available vowels
vowels = {'2', '9', '@', 'E', 'E/', 'O', 'O/', 'U~/', 'a', 'a~', 'e', 'i', 'o', 'o~', 'u', 'y'}

# Maps a speaker to their gender (f or m) and their pronunciation of /E/ in open syllables (/e/ or /E/)
info_dict = {
    'ac1': 'fe',
    'ac2': 'fe',
    'agnes1': 'fE',
    'agnes2': 'fE',
    'al1': 'fe',
    'al2': 'fe',
    'al': 'fe',
    'ang1': 'fe',
    'ang2': 'fe',
    'be1': 'mE',
    'be2': 'mE',
    'cg1': 'fE',
    'cg2': 'fE',
    'clement': 'me',
    'df': 'mE',
    'dj1': 'mE',
    'dj2': 'me',
    'dorian': 'mE',
    'em1': 'mE',
    'em2': 'mE',
    'ev1': 'mE',
    'ev2': 'mE',
    'hzc1': 'fe',
    'hzc2': 'fe',
    'ib1': 'fe',
    'ib2': 'fe',
    'isabel22': 'fE',
    'isabel2': 'fE',
    'jb1': 'mE',
    'jb2': 'mE',
    'kd': 'me',
    'ken1': 'me',
    'laure': 'fe',
    'lb1': 'fE',
    'lb2': 'fE',
    'lp1': 'me',
    'lp2': 'me',
    'lucas': 'me',
    'mat1': 'mE',
    'mathieu2': 'mE',
    'melanie1': 'fE',
    'melanie2': 'fE',
    'mob': 'me',
    'om1': 'fE',
    'om2': 'fE',
    'or1': 'fE',
    'or2': 'fE',
    'oriane-re2': 'fE',
    'oriane1': 'fE',
    'phil1': 'mE',
    'phil2': 'mE',
    'simon1': 'mE',
    'simon2': 'mE',
    'unk1': 'fe',
    'unk2': 'fe',
    'vb1': 'fE',
    'vb2': 'fE',
    'vc1': 'mE',
    'vc2': 'mE',
    'vg': 'mE',
    'vibl1': 'fE',
    'vibl2': 'fE',
    'ya1': 'mE',
    'ya2': 'mE',
    'yf1': 'me',
    'yf2': 'me',
    'yl': 'mE'
}

def trim_wav(in_wav, out_wav, start, end):
    """
    Crops the sound contained in an input wave file at a given start and end (in seconds),
    and saves the result into an output wave file
    """
    sample_rate, wave_data = wavfile.read(in_wav)
    start_sample = int(start * sample_rate)
    end_sample = int(end * sample_rate)
    wavfile.write(out_wav, sample_rate, wave_data[start_sample:end_sample])

def __hash__(self):
    # Needed to have Interval's in sets
    return hash((self.minTime, self.maxTime, self.mark))

Interval.__hash__ = __hash__

trimmed = False # Choose cropped (vowel only) or uncropped (full recording) sound files

for k, (wav, textgrid) in tqdm(final_corpus.items()):
    # Loop other all speakers
    
    # Try to read transcription files
    try:
        tg = TG.fromFile(root_folder + '/' + textgrid)
    except AttributeError:
        continue
    
    prev_p = '' # The phoneme in the previous recording.
    # The order is always 'l', 'm', 'p', 's', 't', 't1' (/t/ with falling intonation)
    
    two_mode = False # Are we currently in a series with nasal vowels? (written as '2' in output files)
    out_phones = defaultdict(list) # Maps a word-level annotation to the list of corresponding phoneme-level annotations
    gender, E_pron = info_dict[k.replace('_mono', '')]
    if len(tg) == 2: # Make sure that we have two transcription tiers: phoneme-level and word-level
        phontier, wordtier = tg
        for phone in phontier: # For each non-silent phoneme-level annotation
            if phone.mark != '#':
                best_overlap_duration = 0 # Try to find the corresponding word-level annotation...
                best_word = None
                for word in wordtier: # ...by computing the overlap percentage with all word-level annotations
                    overlap_duration = max(0, min(phone.maxTime, word.maxTime) - max(phone.minTime, word.minTime))
                    if overlap_duration > best_overlap_duration:
                        best_overlap_duration = overlap_duration
                        best_word = word
                out_phones[best_word].append(phone)
        for word, phones in out_phones.items(): # For each (word, phonemes) mapping, generate the output filename
            word2 = word.mark # Read the word transcription
            phonemes = '_'.join([phone.mark for phone in phones])
            if phonemes == prev_p: # Only happens if we are on a falling-intonation /t/ (t1)
                phonemes = phonemes[0] + '1' + phonemes[1:] # Encode the phoneme name as such
                word2 += '1' # and the word
                if word.mark == 'tant': # If we are on the first word of the nasal vowel series
                    two_mode = True
            elif two_mode:
                phonemes = phonemes[0] + '2' + phonemes[1:] # If on a nasal vowel, encode the phoneme name as such
                word2 =  word2.replace('1', '') + '2'
                
            min_d = word.minTime
            max_d = word.maxTime

            if phones[1:]: # The phonemes are encoded into `phonemes`, only keep the vowel name for now (second vowel)
                phones = phones[1:]
            elif phones[0].mark not in vowels:
                continue
            
            phone = phones[0]
            saved_phone_mark = phone.mark # Save the vowel name
            
            # Exceptions #1 (people proouncing /e/ as /E/)
            if k in ['cg1', 'cg2', 'ib1', 'ib2', 'mathieu2', 'melanie1', 'melanie2', 'vb1']:
                if any(x in word.mark for x in ['pé', 'les', 'mes', 'ses', 'tes']):
                    phone.mark = 'E'
                    # Exceptions #2 (people proouncing /e/ as /E/ but only in some words)
                    if 'pé' in word.mark and ('cg' in k or 'melanie' in k or 'vb1' in k) or \
                    ('ses' in word.mark or 'tes' in word.mark) and 'mathieu' in k:
                        phone.mark = 'e'
            elif any(x in word.mark for x in ['pé', 'les', 'mes', 'ses', 'tes']): # Standard pronunciation
                phone.mark = 'e'
            if any(x in word.mark for x in ['paix', 'lait', 'mais', 'sait', 'taie']): # Hard-coded pronunciation
                phone.mark = E_pron
            if word.mark == 'sœur': # Astali mis-annotated /s9R/ as /syR/
                phonemes = phonemes.replace('y', '9')
                phone.mark = '9'
            
            # Get rid of schwa and / in initial annotations (old annotations only)
            phonemes = phonemes.replace(saved_phone_mark, phone.mark).replace('/', '').replace('@', '2')
            phone.mark = phone.mark.replace('/', '').replace('@', '2')
            add_path = 'trimmed/' if trimmed else ''
            
            # Get position of vowel in recording...
            phon_start = phone.minTime - min_d
            phon_end = phone.maxTime - min_d
            
            # ...as a percentage of the length of the file
            if not trimmed:
                phon_start /= (max_d - min_d)
                phon_end /= (max_d - min_d)
            # Save to file
            trim_wav(f'{root_folder}/{wav}',
                     f'{root_folder}/extracted/{add_path}{phone.mark}__{gender}__{phon_start:.5f}__{phon_end:.5f}__{phonemes}__{word2}__{wav}',
                     phone.minTime if trimmed else min_d,
                     phone.maxTime if trimmed else max_d)
            prev = phones
            prev_p = phonemes

 78%|███████████████████████████████████████████████████████████████████████████████████▊                        | 52/67 [01:39<00:30,  2.04s/it]/tmp/ipykernel_576/719033392.py:76: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, wave_data = wavfile.read(in_wav)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [02:10<00:00,  1.95s/it]
